In [2]:
import spacy
from spacy.lang.ru.examples import sentences
from spacy import displacy
from pprint import pprint
from collections import defaultdict
from tqdm import tqdm
import pandas as pd
import numpy as np

In [3]:
nlp = spacy.load("ru_core_news_sm")

In [4]:
analisys = []

In [5]:
newsDF = pd.read_csv('news.CSV')
text_news = newsDF['text']
text_news = np.array(text_news)
text_news[2]

'\n                            24 мая исполняется 80 лет поражению Германии в Битве за Атлантику. В этот день гросс-адмирал Дениц прекратил борьбу с конвоями, приказав своим подводникам уйти из Северной Атлантики. Произошло это из-за того, что немецкие «волчьи стаи» несли слишком большие потери. Это стало переломом в подводной войне, шедшей с переменным успехом с 1939 года.\nПоражение неминуемо\nС самого начала Второй мировой войны Германия вела боевые действия против судоходства союзников в Атлантике. Командующий немецким подплавом Карл Дениц в этой битве делал ставку на тактику «волчьей стаи» – атаку конвоя группой лодок. Успехи этот метод стал приносить лишь через год, поскольку поначалу кригсмарине просто не хватало субмарин, чтобы как следует реализовывать эту тактику. Ситуация изменилась осенью 1942-го, когда численность немецких подводных сил в Атлантике выросла до 150 единиц. Тогда-то и началась масштабная охота на конвои.\n"Волчья стая" против конвоя: первый успех "серых волко

In [19]:
doc = nlp(text_news[38])
for s in list(doc.sents)[:2]:
  pprint(list([f'{t.lemma_} ({t.text})' for t in s]))

['\n                             (\n                            )']
['" (")',
 'рособоронэкспорт (Рособоронэкспорт)',
 '" (")',
 'на (на)',
 'международный (международной)',
 'выставка (выставке)',
 'вооружение (вооружений)',
 'idex (IDEX)',
 'в (в)',
 'оаэ (ОАЭ)',
 'показать (покажет)',
 'несколько (несколько)',
 'образец (образцов)',
 'стрелковый (стрелкового)',
 'оружие (оружия)',
 'концерн (концерна)',
 '" (")',
 'калашников (Калашников)',
 '" (")',
 ', (,)',
 'а (а)',
 'также (также)',
 'впервые (впервые)',
 'за (за)',
 'рубеж (рубежом)',
 'представить (представит)',
 'снайперский (снайперскую)',
 'винтовка (винтовку)',
 'orsis (ORSIS)',
 ', (,)',
 'сообщить (сообщили)',
 'в (в)',
 'пресс (пресс)',
 '- (-)',
 'служба (службе)',
 'компания (компании)',
 '. (.)',
 '\n (\n)']


In [21]:
for s in list(doc.sents)[:2]:
  displacy.render(s, style="dep", minify=True, jupyter=True, options={"distance":90})

In [7]:
sent0 = list(doc.sents)[0]
childs0 = list(sent0.root.children)
print(sent0.root, childs0)
# Загнать этот вывод в цикл и проанализировать все текста?
print(childs0[0].lemma_, childs0[0].text, childs0[0].dep_, childs0[0].tag_, '->', list(childs0[0].ancestors)[0])

одобрил [Госдепартамент, продажу, сообщило, .]
госдепартамент Госдепартамент nsubj PROPN -> одобрил


In [6]:
count = 0
for line in tqdm(text_news):
  try:
    doc = nlp(line)
    sent0 = list(doc.sents)[0]
    childs0 = list(sent0.root.children)
    analisys.append((childs0[0].lemma_, childs0[0].text, childs0[0].dep_, childs0[0].tag_, '->', list(childs0[0].ancestors)[0]))
    count+=1
  except KeyboardInterrupt:
    break
  except:
    print(f'Не удалось провести анализ статьи под номером {count}')
    analisys.append(None)
    count+=1
  

  2%|▏         | 41/1775 [00:09<04:17,  6.74it/s]

Не удалось провести анализ статьи под номером 38
Не удалось провести анализ статьи под номером 41


 11%|█         | 190/1775 [00:26<02:22, 11.13it/s]

Не удалось провести анализ статьи под номером 187


 14%|█▍        | 246/1775 [00:31<02:21, 10.79it/s]

...

 94%|█████████▎| 1663/1775 [03:11<00:10, 10.84it/s]

Не удалось провести анализ статьи под номером 1662


 96%|█████████▌| 1708/1775 [03:16<00:06,  9.84it/s]

Не удалось провести анализ статьи под номером 1707


100%|██████████| 1775/1775 [03:24<00:00,  8.69it/s]


In [7]:
analisys

[('полугодие', 'полугодии', 'obl', 'NOUN', '->', рассчитывает),
 ('поставка', 'Поставки', 'nsubj', 'NOUN', '->', начались),
 ('24', '24', 'obl', 'ADJ', '->', исполняется),
 ('россия', 'России', 'obl', 'PROPN', '->', продолжаются),
 ('22', '22', 'obl', 'ADJ', '->', исполняется),
 ('вооружение', 'вооружение', 'obl', 'NOUN', '->', поступил),
 ('россия', 'России', 'obl', 'PROPN', '->', начали),
 ('цифровизация', 'Цифровизация', 'nsubj', 'NOUN', '->', происходит),
 ('война', 'война', 'nsubj', 'NOUN', '->', складывалась),
 ('конец', 'конце', 'obl', 'NOUN', '->', принят),
 ('командование', 'Командование', 'nsubj', 'NOUN', '->', приказало),
 ('госдепартамент', 'Госдепартамент', 'nsubj', 'PROPN', '->', одобрил),
 ('24', '24', 'obl', 'ADJ', '->', покончил),
 ('лето', 'Летом', 'obl', 'NOUN', '->', заявили),
 ('власть', 'Власти', 'nsubj', 'NOUN', '->', планируют),
 ('18', '18', 'obl', 'ADJ', '->', исполняется),
 ('рубеж', 'рубеже', 'obl', 'NOUN', '->', третьим),
 ('инженер', 'инженеры', 'nsubj', '

In [8]:
analisys_result = newsDF
analisys_result['syntaxAnalisys'] = analisys
analisys_result

,href,title,text,theme,syntaxAnalisys
0,https://profile.ru/news/military/kalashnikov-v...,"""Калашников"" выпустит партию обновленных АК-12...",\n Во втором полуго...,0.0,"(полугодие, полугодии, obl, NOUN, ->, рассчиты..."
1,https://profile.ru/news/military/kalashnikov-n...,"""Калашников"" начал поставлять снайперскую винт...",\n Поставки новейше...,0.0,"(поставка, Поставки, nsubj, NOUN, ->, начались)"
2,https://profile.ru/military/chernyj-maj-kak-ne...,"""Черный май"": как немецкие подводники потерпел...",\n 24 мая исполняет...,0.0,"(24, 24, obl, ADJ, ->, исполняется)"
3,https://profile.ru/news/military/v-rf-prodolzh...,В РФ продолжают испытывать новый ракетный комп...,\n В России продолж...,0.0,"(россия, России, obl, PROPN, ->, продолжаются)"
4,https://profile.ru/military/za-chto-komandir-s...,"За что командир советской подлодки ""Щ-408"" пол...",\n 22 мая исполняет...,0.0,"(22, 22, obl, ADJ, ->, исполняется)"
...,...,...,...,...,...
1770,https://profile.ru/news/dk/ugmk/shaaz-vypustil...,ШААЗ выпустил первый отечественный 14-тонный п...,\n «Шадринский авто...,3.0,"(завод, завод, nsubj, NOUN, ->, выпустил)"
1771,https://profile.ru/news/dk/rosterminalugol/dey...,Деятельность «Ростерминалугля» признана эколог...,\n «Ростерминалугол...,3.0,"(ростерминалуголь, Ростерминалуголь, nsubj, NO..."
1772,https://profile.ru/dk/ugmk/perspektiva-dlya-za...,«Перспектива» для заслуженного отдыха,\n Почему у нашего ...,3.0,"(почему, Почему, advmod, ADV, ->, развита)"
1773,https://profile.ru/news/dk/kru/kuzbassrazrezug...,«Кузбассразрезуголь» создал первый профстандар...,\n «Кузбассразрезуг...,3.0,"(кузбассразрезуголь, Кузбассразрезуголь, nsubj..."


In [9]:
analisys_result.to_csv('analisys_result.csv', index=False)

In [7]:
with open("rusentilex_2017.txt", encoding="utf-8") as senti_file: 
    for i in range(18): # Пропускаем заголовок файла.
        _ = senti_file.readline()
        
    senti_words = {line.split(", ")[0]: line.split(", ")[3] for line in senti_file.readlines()}

In [8]:
def find_sentiments_spacy(node, senti_words, sentiments):
    for child in node.children:
        if child.lemma_ in senti_words.keys():
            if node.lemma_ not in sentiments.keys():
                sentiments[node.lemma_] = defaultdict(int)
            sentiments[node.lemma_][senti_words[child.lemma_]] += 1

In [9]:
sentiments = {}
for sent in doc.sents:
    find_sentiments_spacy(sent.root, senti_words, sentiments)
sentiments

{}

In [10]:
sentiments = {}
for sent_text in tqdm(text_news):
    doc = nlp(sent_text)
    for sentence in doc.sents:
        find_sentiments_spacy(sentence.root, senti_words, sentiments)

100%|██████████| 1775/1775 [03:51<00:00,  7.66it/s]


In [12]:
[(senti, vals) for senti,vals in sentiments.items() if vals['negative']>30], '\n----\n',\
[(senti, vals) for senti,vals in sentiments.items() if vals['positive']>30]

([('стать',
   defaultdict(int,
               {'positive': 153,
                'пострадать': 1,
                'negative': 100,
                'neutral': 54,
                'positive/negative': 6})),
  ('нанести',
   defaultdict(int,
               {'negative': 32,
                'positive': 3,
                'positive/negative': 3,
                'neutral': 1})),
  ('мочь',
   defaultdict(int,
               {'positive/negative': 17,
                'negative': 57,
                'neutral': 33,
                'positive': 83,
                'пострадать': 1})),
  ('оказаться',
   defaultdict(int,
               {'negative': 54,
                'positive': 51,
                'neutral': 18,
                'positive/negative': 3})),
  ('получить',
   defaultdict(int,
               {'positive': 36,
                'negative': 33,
                'positive/negative': 1,
                'neutral': 8}))],
 '\n----\n',
 [('стать',
   defaultdict(int,
               {'positive': 15